<a href="https://colab.research.google.com/github/rodannemann/imersaoAI2024/blob/main/ImersaoAI2024_RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projeto ImersãoAI2024-RD
#### Classificação Inteligente de Posts em Mídias Sociais para Maximização de Receitas.
#### Um Estudo de Caso com um Grande Time de Futebol Brasileiro

## Inicializando o Modelo AI

In [1]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

from pathlib import Path
import hashlib
import google.generativeai as genai

# código para ocultar a chave de API
from google.colab import userdata
api_key_rod = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key_rod)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


#### Leitura das Bases de Dados de Treino e Testes

In [5]:
import os
#os.chdir("C:\\Users\\rdann\\Jornada Python")
os.getcwd()

'/content'

In [4]:
import numpy as np
import pandas as pd

# Carrega a base de treino
base_treino = pd.read_csv('data/crf_teste.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/crf_teste.csv'

## Treinando o Modelo

In [2]:
prompt_parts = [
  "input: ",
  genai.upload_file("<path>/image0.png"),
  "input 2: Flamengo | Fim de jogo no Maracanã. O Flamengo é derrotado pelo Fluminense por 2 a 0 na primeira partida da final do Campeonato Carioca. \n\n#CRF \n\n... | Instagram",
  "output: 1",
  "input: ",
  genai.upload_file("<path>/image1.png"),
  "input 2: Flamengo | Final de jogo no Mané Garrincha: \n\nFlamengo 0x1 Botafogo #CRF \n\n📸: @paulareis.raw / CRF | Instagram",
  "output: 1",
  "input: ",
  genai.upload_file("<path>/image2.png"),
  "input 2: Flamengo | FIMMMMMMMMMMMMM DE JOGO NO MARACANÃ! MENGÃO GOLEIA O TOLIMA POR 7 A 1 COM GOLS DE PEDRO (4), GABIGOL, MATHEUS FRANÇA E QUIÑONES (CONT... | Instagram",
  "output: 1",
  "input: ",
  genai.upload_file("<path>/image3.png"),
  "input 2: Flamengo | FIMMMMMMMMMMMMMMMMM DE JOGO NO MARACANÃ! O MENGÃO VENCE O ATLÉTICO-MG POR 2 A 0 COM GOLS DE ARRASCAETA E SE CLASSIFICA PARA AS QUARTAS... | Instagram",
  "output: 1",
  "input: ",
  genai.upload_file("<path>/image4.png"),
  "input 2: Flamengo | FIMMMMMMMMMMMMM DE JOGO NO MORUMBI! O MENGÃO VENCE O SÃO PAULO POR 3 A 1 COM GOLS DE JOÃO GOMES, GABIGOL E EVERTON CEBOLINHA NO JOGO D... | Instagram",
  "output: ",
]

response = model.generate_content(prompt_parts)
print(response.text)

FileNotFoundError: [Errno 2] No such file or directory: '<path>/image0.png'